## **INSTRUÇÕES**

---
### Todos os arquivos baixados devem ser upados na pasta 'Arquivos de entrada'
---

#### **Passo 1:**  Gere as planilhas BID US Futuro.csv e BID BR Futuro.csv, nas quais constam apenas os SKUs dos hotéis e os valores de bid:

* BID US Futuro.csv
* BID BR Futuro.csv

#### **Passo 3:** Acesse o SFTP do Trivago, vá para a página outfiles e baixe o arquivo **feed_cpa.csv**. Não é necessário renomeá-lo:

* feed_cpa.csv

---
### Todos os arquivos baixados para essa seção devem ser upados na pasta Bid Tuner Utils
---

#### **Passo 3:** Acesse o SFTP do Trivago, vá para a página outfiles e baixe o arquivo **feed_cpa.csv**. Não é necessário renomeá-lo. Apenas coloque-o na pasta de arquivos de entrada:

* feed_cpa.csv

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
import pandas as pd
import datetime as dt

In [3]:
# Importing
base_bids_br = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/BID BR Futuro.csv', delimiter = ';',encoding='ISO 8859-1')).fillna(0)
base_bids_us = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/BID US Futuro.csv', delimiter = ';',encoding='ISO 8859-1')).fillna(0)
skus_mvp = pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/bid tuner utils/sku_list.csv', usecols = ['sku'])
campaigns = pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/feed_cpa.csv', delimiter = ',', encoding="ISO 8859-1")


In [4]:
# Cleaning names
base_bids_br.rename(columns = {'ï»¿sku':'sku'}, inplace = True)
base_bids_us.rename(columns = {'ï»¿sku':'sku'}, inplace = True)


In [5]:
skus_mvp

,sku
0,OMN-8351
1,OMN-10179
2,OMN-3925
3,NHU-8421
4,OMN-5902
...,...
571,OMN-2770
572,OMN-1320
573,OMN-2449
574,OMN-6388


In [6]:
campaigns.head()

,locale,partner_reference,campaign
0,BR,JNP-JP984417,0
1,BR,OMN-142,0
2,BR,OMN-4789,0
3,BR,OMN-4811,0
4,BR,OMN-4814,0


In [7]:
base_bids_br

,sku,BR
0,OMN-4273,3
1,OMN-3185,15
2,JNP-JP408570,16
3,OMN-4480,5
4,JNP-JP335495,54
...,...,...
108678,JNP-JP077769,67
108679,JNP-JP077766,67
108680,JNP-JP077761,0
108681,JNP-JP077757,67


In [8]:
base_bids_us

,sku,US
0,JNP-JP06450R,17
1,JNP-JP055209,15
2,JNP-JP312267,40
3,JNP-JP981787,40
4,JNP-JP735105,15
...,...,...
108688,JNP-JP074552,15
108689,JNP-JP074544,49
108690,JNP-JP074521,17
108691,JNP-JP074515,49


In [9]:
skus_mvp

,sku
0,OMN-8351
1,OMN-10179
2,OMN-3925
3,NHU-8421
4,OMN-5902
...,...
571,OMN-2770
572,OMN-1320
573,OMN-2449
574,OMN-6388


In [10]:
# Creating list of deprecated skus and removing then from the mvp list
deprecated_skus = ["OMN-6147",    
                   "OMN-5984",
                   "OMN-7039",
                   "OMN-3035",
                   "OMN-5436",
                   "OMN-3185",
                   "OMN-2317"]

skus_mvp = skus_mvp.loc[~skus_mvp['sku'].isin(deprecated_skus)]

In [11]:
skus_mvp

,sku
0,OMN-8351
1,OMN-10179
2,OMN-3925
3,NHU-8421
4,OMN-5902
...,...
571,OMN-2770
572,OMN-1320
573,OMN-2449
574,OMN-6388


In [12]:
mvp_bids_br = base_bids_br.merge(skus_mvp, how = 'inner', on = 'sku')
mvp_bids_us = base_bids_us.merge(skus_mvp, how = 'inner', on = 'sku')

In [13]:
def noise_bid(df, pos, floor, ceiling):
    """
    Função usada para testagem de bids adjacentes ao bid dado no dia.
    Ela retornará um novo dataframe com as linhas do dataframe original
    de forma duplicada com valores de bid que variam entre bid + floor e 
    bid + ceiling. Para correto funcionamento, usar um floor negativo e 
    um ceiling positivo, de forma que 0 sempre pertença ao intervalo.
    
    """   
    df_list = list()
    for inc in range(floor, ceiling + 1):
        temp_df = df.copy()
        temp_df[pos] += inc
        df_list.append(temp_df)
    return pd.concat(objs = df_list).reset_index(drop = True)

In [14]:
### Removing hotels in CPA campaigns and adding noise.

# Defining variables
cpc_campaigns_br = campaigns.query('(campaign == 0) & (locale == "BR")')
cpc_campaigns_us = campaigns.query('(campaign == 0) & (locale == "US")')
mvp_bids_br = mvp_bids_br[mvp_bids_br['sku'].isin(cpc_campaigns_br['partner_reference'])]
mvp_bids_us = mvp_bids_us[mvp_bids_us['sku'].isin(cpc_campaigns_us['partner_reference'])]

# checking mvp_bids_br shape
display('Shape do mvp_bids_br: {}'.format(mvp_bids_br.shape))
display('Shape do mvp_bids_us: {}'.format(mvp_bids_us.shape))

# Adding noise
output_br = noise_bid(mvp_bids_br, 'BR', -1, 1)
output_us = noise_bid(mvp_bids_us, 'US', -1, 1)

# Checking output_br shape
display('Shape do output br: {}'.format(output_br.shape))
display('Shape do output us: {}'.format(output_us.shape))

output_br['BR'] = output_br['BR']/100 
output_us['US'] = output_us['US']/100

output_br.drop(labels = output_br[output_br['BR'] <= 0].index, inplace = True)
output_us.drop(labels = output_us[output_us['US'] <= 0].index, inplace = True)

display('Shape do output br: {}'.format(output_br.shape))
display('Shape do output us: {}'.format(output_us.shape))

output_br = output_br.sort_values(by = 'sku')
output_us = output_us.sort_values(by = 'sku')


'Shape do mvp_bids_br: (285, 2)'

'Shape do mvp_bids_us: (116, 2)'

'Shape do output br: (855, 2)'

'Shape do output us: (348, 2)'

'Shape do output br: (853, 2)'

'Shape do output us: (346, 2)'

In [15]:
display('Shape do output br: {}'.format(output_br.shape))
display('Shape do output us: {}'.format(output_us.shape))

'Shape do output br: (853, 2)'

'Shape do output us: (346, 2)'

In [16]:
# CUSPINDO PLANILHA FINAL
dia = (dt.datetime.today() + dt.timedelta(days=1)).strftime(format='%d_%m_%y')

# Cospe tabela BR
output_br.drop_duplicates(inplace = True)
output_br.to_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/bid tuner utils/bid_tuner_BR_' + dia + '.csv', sep=',', decimal='.', 
                              encoding="utf-8-sig", index = False, columns = ['sku', 'BR'])

output_us.drop_duplicates(inplace = True)
output_us.to_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/bid tuner utils/bid_tuner_US_' + dia + '.csv', sep=',', decimal='.', 
                              encoding="utf-8-sig", index = False, columns = ['sku', 'US'])

# Cospe tabela dos skus atualmente utilizados no MVP
mvp_bids_br.to_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/bid tuner utils/skus_mvp_test.csv', columns = ['sku'], index = False)


In [17]:
output_br['BR'].unique()

array([0.78, 0.77, 0.76, 0.73, 0.71, 0.72, 0.32, 0.3 , 0.31, 0.57, 0.59,
       0.58, 0.29, 0.28, 0.64, 0.65, 0.63, 0.84, 0.85, 0.83, 0.36, 0.34,
       0.35, 0.49, 0.48, 0.47, 0.26, 0.27, 0.52, 0.53, 0.51, 0.55, 0.54,
       0.45, 0.44, 0.43, 0.5 , 0.61, 0.6 , 0.42, 0.41, 0.37, 0.67, 0.66,
       0.68, 0.56, 0.62, 0.4 , 0.93, 0.92, 0.94, 0.46, 1.05, 1.03, 1.04,
       0.82, 0.96, 0.97, 0.95, 0.69, 0.25, 0.24, 0.7 , 1.08, 1.06, 1.07,
       0.99, 0.98, 0.21, 0.2 , 0.22, 1.02, 0.38, 0.39, 0.74, 0.75, 0.79,
       0.8 , 0.81, 1.31, 1.32, 1.3 , 0.33, 1.1 , 1.12, 1.11, 1.26, 1.27,
       1.28, 0.15, 0.13, 0.14, 0.23, 0.01, 0.18, 0.16, 0.17, 1.25, 1.24,
       1.19, 1.21, 1.2 , 1.09, 1.01, 1.  , 0.9 , 0.91, 0.88, 0.89, 0.87,
       0.86, 0.19])

In [18]:
cpc_campaigns_br = campaigns.query('(campaign == 0) & (locale == "BR")')

In [19]:
mvp_bids_br

,sku,BR
1,OMN-6025,57
2,OMN-8598,39
3,OMN-4609,72
7,OMN-2087,21
8,OMN-2081,49
...,...,...
481,OMN-5591,73
492,OMN-5465,17
497,OMN-5718,47
500,OMN-5739,44


In [20]:
output_br

,sku,BR
799,OMN-1009,0.78
514,OMN-1009,0.77
229,OMN-1009,0.76
608,OMN-10179,0.73
38,OMN-10179,0.71
...,...,...
356,OMN-989,0.48
71,OMN-989,0.47
87,OMN-9899,0.45
372,OMN-9899,0.46
